## 05 DPA sex specific


1) differential proportion analysis - MPPs and LTs male and female

from this paper https://elifesciences.org/articles/43882#s4  

done with this docker image:

docker run --rm -d --name test_eva -p 8883:8888 -e JUPYTER_ENABLE_LAB=YES -v /Users/efast/Documents/:/home/jovyan/work r_scanpy:vs5

In [18]:
#load some packages that are needed for plotting + computation

import math
from matplotlib import pyplot as plt
import pandas as pd
import scipy.stats as stats
import seaborn as sns
%matplotlib inline

## 2) Differential proportion analysis

In [19]:
import scanpy as sc
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sb
import scipy.stats as stats

import rpy2.rinterface_lib.callbacks
import logging

from rpy2.robjects import pandas2ri
import anndata2ri

In [20]:
# Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

# Automatically convert rpy2 outputs to pandas dataframes
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3
sc.logging.print_versions()

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
scanpy==1.4.5.1 anndata==0.7.1 umap==0.3.10 numpy==1.17.3 scipy==1.3.0 pandas==0.25.3 scikit-learn==0.22.2.post1 statsmodels==0.10.0 python-igraph==0.7.1 louvain==0.6.1


In [21]:
%%R
# Load libraries from correct lib Paths for my environment - ignore this!
.libPaths(.libPaths()[c(3,2,1)])

# Load all the R libraries we will be using in the notebook
library(ggplot2)
library(plyr)

source("diffprop_functions.R");

## Comparison LT clusters

In [22]:
# read csvs, split into treatments, rename columns, merge and transpose
LT = pd.read_csv('./write/LT_cell_numbers_sex.csv', index_col = 0)
#LT[LT < 5] = 0
LT = LT.T
LT

leiden,Metabolism,Quiescent,Activated,Interferon,Acute-Activation,Cell-cycle
GCSF_f,804.0,625.0,502.0,15.0,0.0,30.0
ct_f,1051.0,1079.0,915.0,20.0,0.0,50.0
dmPGE2_f,6.0,7.0,19.0,0.0,62.0,0.0
indo_f,853.0,830.0,931.0,31.0,0.0,20.0
pIC_f,106.0,102.0,25.0,196.0,0.0,1.0
GCSF_m,400.0,312.0,278.0,6.0,0.0,12.0
ct_m,814.0,902.0,641.0,20.0,0.0,29.0
dmPGE2_m,77.0,17.0,89.0,0.0,199.0,0.0
indo_m,904.0,986.0,1016.0,27.0,1.0,27.0
pIC_m,82.0,82.0,33.0,120.0,0.0,1.0


In [23]:
%%R -i LT
## Read in file of counts of cells in each population across conditions

obs.counts <- as.matrix(LT_new)
print(obs.counts)

         Metabolism Quiescent Activated Interferon Acute.Activation Cell.cycle
GCSF_f          407       316       254          8                0         15
ct_f            337       346       294          6                0         16
dmPGE2_f         64        74       202          0              660          0
indo_f          320       311       349         12                0          8
pIC_f           247       238        58        457                0          0
GCSF_m          397       310       276          6                0         12
ct_m            338       375       266          8                0         12
dmPGE2_m        202        45       233          0              521          0
indo_m          305       333       343          9                0          9
pIC_m           259       259       104        379                0          0


In [24]:
%%R

## Run an example using error (p) of 0.1 and with 100,000 iterations
tip.exp <- generateNull(obs.counts, n=100000, p=0.1);     # Generate the null distribution based on sampling

[1] 1000
[1] 2000
[1] 3000
[1] 4000
[1] 5000
[1] 6000
[1] 7000
[1] 8000
[1] 9000
[1] 10000
[1] 11000
[1] 12000
[1] 13000
[1] 14000
[1] 15000
[1] 16000
[1] 17000
[1] 18000
[1] 19000
[1] 20000
[1] 21000
[1] 22000
[1] 23000
[1] 24000
[1] 25000
[1] 26000
[1] 27000
[1] 28000
[1] 29000
[1] 30000
[1] 31000
[1] 32000
[1] 33000
[1] 34000
[1] 35000
[1] 36000
[1] 37000
[1] 38000
[1] 39000
[1] 40000
[1] 41000
[1] 42000
[1] 43000
[1] 44000
[1] 45000
[1] 46000
[1] 47000
[1] 48000
[1] 49000
[1] 50000
[1] 51000
[1] 52000
[1] 53000
[1] 54000
[1] 55000
[1] 56000
[1] 57000
[1] 58000
[1] 59000
[1] 60000
[1] 61000
[1] 62000
[1] 63000
[1] 64000
[1] 65000
[1] 66000
[1] 67000
[1] 68000
[1] 69000
[1] 70000
[1] 71000
[1] 72000
[1] 73000
[1] 74000
[1] 75000
[1] 76000
[1] 77000
[1] 78000
[1] 79000
[1] 80000
[1] 81000
[1] 82000
[1] 83000
[1] 84000
[1] 85000
[1] 86000
[1] 87000
[1] 88000
[1] 89000
[1] 90000
[1] 91000
[1] 92000
[1] 93000
[1] 94000
[1] 95000
[1] 96000
[1] 97000
[1] 98000
[1] 99000
[1] 100000


In [25]:
%%R
### P-value tests for old vs new
print('ct')
print(two.class.test(obs.counts, tip.exp, cond.control="ct_f", cond.treatment="ct_m",to.plot=F))
print('GCSF')
print(two.class.test(obs.counts, tip.exp, cond.control="GCSF_f", cond.treatment="GCSF_m",to.plot=F))
print('dmPGE2')
print(two.class.test(obs.counts, tip.exp, cond.control="dmPGE2_f", cond.treatment="dmPGE2_m",to.plot=F))
print('pIC')
print(two.class.test(obs.counts, tip.exp, cond.control="pIC_f", cond.treatment="pIC_m",to.plot=F))
print('indo')
print(two.class.test(obs.counts, tip.exp, cond.control="indo_f", cond.treatment="indo_m",to.plot=F))

[1] "ct"
      Metabolism        Quiescent        Activated       Interferon 
         0.48423          0.38489          0.36618          0.45091 
Acute.Activation       Cell.cycle 
         0.45699          0.33984 
[1] "GCSF"
      Metabolism        Quiescent        Activated       Interferon 
         0.43743          0.45208          0.41064          0.47376 
Acute.Activation       Cell.cycle 
         0.45682          0.35563 
[1] "dmPGE2"
      Metabolism        Quiescent        Activated       Interferon 
         0.03853          0.37151          0.34336          0.45495 
Acute.Activation       Cell.cycle 
         0.07954          0.31497 
[1] "pIC"
      Metabolism        Quiescent        Activated       Interferon 
         0.43328          0.39613          0.24813          0.17636 
Acute.Activation       Cell.cycle 
         0.46050          0.31362 
[1] "indo"
      Metabolism        Quiescent        Activated       Interferon 
         0.44347          0.38157          0.

## Comparison MPP clusters

In [26]:
# read csvs, split into treatments, rename columns, merge and transpose
MPP = pd.read_csv('./write/MPP_cell_numbers_sex.csv', index_col = 0)
MPP = MPP.T
MPP

leiden,Primed,Metabolism,Stem cell,Cell-cycle,Acute-Activation,Interferon,Interferon cell-cycle,Myeloid
ct_f,120.0,109.0,144.0,73.0,4.0,2.0,0.0,0.0
indo_f,237.0,230.0,222.0,145.0,2.0,9.0,1.0,2.0
GCSF_f,292.0,259.0,258.0,149.0,6.0,3.0,1.0,6.0
dmPGE2_f,10.0,161.0,57.0,190.0,592.0,1.0,0.0,21.0
pIC_f,234.0,118.0,102.0,96.0,0.0,400.0,110.0,15.0
ct_m,126.0,114.0,93.0,75.0,4.0,0.0,0.0,2.0
indo_m,222.0,155.0,166.0,91.0,3.0,4.0,0.0,2.0
GCSF_m,257.0,199.0,162.0,102.0,4.0,1.0,0.0,3.0
dmPGE2_m,21.0,173.0,83.0,183.0,481.0,0.0,0.0,19.0
pIC_m,205.0,197.0,144.0,115.0,0.0,313.0,86.0,5.0


In [27]:
%%R -i MPP
## Read in file of counts of cells in each population across conditions

obs.counts <- as.matrix(MPP_new)
print(obs.counts)

         Primed Metabolism Stem.cell Cell.cycle Acute.Activation Interferon
ct_f        269        244       323        164                0          0
indo_f      281        273       263        172                0         11
GCSF_f      301        267       266        154                6          0
dmPGE2_f     10        156        55        184              574          0
pIC_f       218        110        95         89                0        372
ct_m        309        279       228        184                0          0
indo_m      350        244       262        144                0          0
GCSF_m      357        276       225        142                0          0
dmPGE2_m     22        180        86        191              501          0
pIC_m       192        185       135        108                0        294
         Interferon.cell.cycle Myeloid
ct_f                         0       0
indo_f                       0       0
GCSF_f                       0       6
dmPGE2_f

In [28]:
%%R

## Run an example using error (p) of 0.05 and with 100,000 iterations
tip.exp <- generateNull(obs.counts, n=100000, p=0.1);     # Generate the null distribution based on sampling

[1] 1000
[1] 2000
[1] 3000
[1] 4000
[1] 5000
[1] 6000
[1] 7000
[1] 8000
[1] 9000
[1] 10000
[1] 11000
[1] 12000
[1] 13000
[1] 14000
[1] 15000
[1] 16000
[1] 17000
[1] 18000
[1] 19000
[1] 20000
[1] 21000
[1] 22000
[1] 23000
[1] 24000
[1] 25000
[1] 26000
[1] 27000
[1] 28000
[1] 29000
[1] 30000
[1] 31000
[1] 32000
[1] 33000
[1] 34000
[1] 35000
[1] 36000
[1] 37000
[1] 38000
[1] 39000
[1] 40000
[1] 41000
[1] 42000
[1] 43000
[1] 44000
[1] 45000
[1] 46000
[1] 47000
[1] 48000
[1] 49000
[1] 50000
[1] 51000
[1] 52000
[1] 53000
[1] 54000
[1] 55000
[1] 56000
[1] 57000
[1] 58000
[1] 59000
[1] 60000
[1] 61000
[1] 62000
[1] 63000
[1] 64000
[1] 65000
[1] 66000
[1] 67000
[1] 68000
[1] 69000
[1] 70000
[1] 71000
[1] 72000
[1] 73000
[1] 74000
[1] 75000
[1] 76000
[1] 77000
[1] 78000
[1] 79000
[1] 80000
[1] 81000
[1] 82000
[1] 83000
[1] 84000
[1] 85000
[1] 86000
[1] 87000
[1] 88000
[1] 89000
[1] 90000
[1] 91000
[1] 92000
[1] 93000
[1] 94000
[1] 95000
[1] 96000
[1] 97000
[1] 98000
[1] 99000
[1] 100000


In [29]:
%%R
### P-value tests for old vs new
print('ct')
print(two.class.test(obs.counts, tip.exp, cond.control="ct_f", cond.treatment="ct_m",to.plot=F))
print('GCSF')
print(two.class.test(obs.counts, tip.exp, cond.control="GCSF_f", cond.treatment="GCSF_m",to.plot=F))
print('dmPGE2')
print(two.class.test(obs.counts, tip.exp, cond.control="dmPGE2_f", cond.treatment="dmPGE2_m",to.plot=F))
print('pIC')
print(two.class.test(obs.counts, tip.exp, cond.control="pIC_f", cond.treatment="pIC_m",to.plot=F))
print('indo')
print(two.class.test(obs.counts, tip.exp, cond.control="indo_f", cond.treatment="indo_m",to.plot=F))

[1] "ct"
               Primed            Metabolism             Stem.cell 
              0.29962               0.31606               0.11302 
           Cell.cycle      Acute.Activation            Interferon 
              0.36449               0.45612               0.44277 
Interferon.cell.cycle               Myeloid 
              0.38877               0.29973 
[1] "GCSF"
               Primed            Metabolism             Stem.cell 
              0.25183               0.43397               0.28088 
           Cell.cycle      Acute.Activation            Interferon 
              0.40501               0.42403               0.44398 
Interferon.cell.cycle               Myeloid 
              0.39110               0.18851 
[1] "dmPGE2"
               Primed            Metabolism             Stem.cell 
              0.40253               0.36083               0.30017 
           Cell.cycle      Acute.Activation            Interferon 
              0.48014               0.21525       

In [13]:
sc.logging.print_versions()
pd.show_versions()

scanpy==1.4.5.1 anndata==0.7.1 umap==0.3.10 numpy==1.17.3 scipy==1.3.0 pandas==0.25.3 scikit-learn==0.22.2.post1 statsmodels==0.10.0 python-igraph==0.7.1 louvain==0.6.1

INSTALLED VERSIONS
------------------
commit           : None
python           : 3.7.3.final.0
python-bits      : 64
OS               : Linux
OS-release       : 4.19.76-linuxkit
machine          : x86_64
processor        : x86_64
byteorder        : little
LC_ALL           : en_US.UTF-8
LANG             : en_US.UTF-8
LOCALE           : en_US.UTF-8

pandas           : 0.25.3
numpy            : 1.17.3
pytz             : 2019.3
dateutil         : 2.8.1
pip              : 19.3.1
setuptools       : 41.6.0.post20191101
Cython           : None
pytest           : 5.3.5
hypothesis       : None
sphinx           : None
blosc            : None
feather          : None
xlsxwriter       : None
lxml.etree       : None
html5lib         : None
pymysql          : None
psycopg2         : None
jinja2           : 2.10.3
IPython          : 7.

In [14]:
!pip list

Package                Version            
---------------------- -------------------
alembic                1.3.0              
anndata                0.7.1              
anndata2ri             1.0.2              
async-generator        1.10               
attrs                  19.3.0             
backcall               0.1.0              
bleach                 3.1.0              
blinker                1.4                
certifi                2019.11.28         
certipy                0.1.3              
cffi                   1.13.2             
chardet                3.0.4              
conda                  4.7.12             
conda-package-handling 1.6.0              
cryptography           2.8                
cycler                 0.10.0             
decorator              4.4.1              
defusedxml             0.6.0              
entrypoints            0.3                
get-version            2.1                
gprofiler-official     1.0.0              
h5py       

In [15]:
sc.logging.print_versions()
pd.show_versions()

scanpy==1.4.5.1 anndata==0.7.1 umap==0.3.10 numpy==1.17.3 scipy==1.3.0 pandas==0.25.3 scikit-learn==0.22.2.post1 statsmodels==0.10.0 python-igraph==0.7.1 louvain==0.6.1

INSTALLED VERSIONS
------------------
commit           : None
python           : 3.7.3.final.0
python-bits      : 64
OS               : Linux
OS-release       : 4.19.76-linuxkit
machine          : x86_64
processor        : x86_64
byteorder        : little
LC_ALL           : en_US.UTF-8
LANG             : en_US.UTF-8
LOCALE           : en_US.UTF-8

pandas           : 0.25.3
numpy            : 1.17.3
pytz             : 2019.3
dateutil         : 2.8.1
pip              : 19.3.1
setuptools       : 41.6.0.post20191101
Cython           : None
pytest           : 5.3.5
hypothesis       : None
sphinx           : None
blosc            : None
feather          : None
xlsxwriter       : None
lxml.etree       : None
html5lib         : None
pymysql          : None
psycopg2         : None
jinja2           : 2.10.3
IPython          : 7.

In [17]:
%%R

sessionInfo()

R version 3.6.1 (2019-07-05)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: Ubuntu 18.04.2 LTS

Matrix products: default
BLAS/LAPACK: /opt/conda/lib/libopenblasp-r0.3.7.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] tools     stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
[1] plyr_1.8.4    ggplot2_3.2.1

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.3       withr_2.1.2      crayon_1.3.4     dplyr_0.8.3     
 [5] assertthat_0.2.1 grid_3.6.1       R6_2.4.0         gtable_0.3.0    
 [9] magrittr_1.5     scales_1.0.0     pillar_1.4.2     rlang_0.4.1     
[13] lazyeval_0.